In [1]:
import numpy as np
import pandas as pd

In [4]:
# load tsne glove embeddings from csv
tsne_df = pd.read_csv('./data/tsne_glove.csv')
tsne_df

,word,x,y
0,the,85.077200,9.514782
1,",",84.960620,9.250578
2,.,85.001900,9.201346
3,of,84.726130,9.356920
4,to,85.887340,7.373219
...,...,...,...
399995,chanty,21.977028,6.534129
399996,kronik,3.715976,-25.502014
399997,rolonda,-8.363313,-2.412165
399998,zsombor,-32.306004,-27.756212


In [6]:
# translate dataframe to json
tsne_df.to_json('./data/tsne_glove.json', orient='records')